In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Dropout, Input, concatenate
from keras.utils import to_categorical


2023-04-02 11:22:23.379442: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 11:22:25.551051: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-02 11:22:25.551088: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-02 11:22:31.025912: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# Load data from signal.feather file
df_1 = pd.read_feather('/home/brandon/Omdena/Omdena HeartKinetics/HeartKinetics - Project/signals.feather')
df_2 = pd.read_feather('/home/brandon/Omdena/Omdena HeartKinetics/HeartKinetics - Project/records.feather')

In [3]:
df_1.head()

,record_id,channel,0 days 00:00:00,0 days 00:00:00.005000,0 days 00:00:00.010000,0 days 00:00:00.015000,0 days 00:00:00.020000,0 days 00:00:00.025000,0 days 00:00:00.030000,0 days 00:00:00.035000,...,0 days 00:00:59.955000,0 days 00:00:59.960000,0 days 00:00:59.965000,0 days 00:00:59.970000,0 days 00:00:59.975000,0 days 00:00:59.980000,0 days 00:00:59.985000,0 days 00:00:59.990000,0 days 00:00:59.995000,0 days 00:01:00
0,Frontiers_CP-66_CP-66_20210716-000000,nrg_lin,1.279607e-10,8.848643e-10,2.081432e-09,3.121196e-09,3.971332e-09,4.824464e-09,5.844231e-09,7.626334e-09,...,3.843697e-08,4.362340e-08,4.407751e-08,4.144375e-08,3.601437e-08,2.377446e-08,1.072449e-08,4.133584e-09,1.771588e-09,2.260352e-09
1,Frontiers_CP-66_CP-66_20210716-000000,nrg_rot,5.556937e-11,2.217992e-10,3.255192e-10,5.954308e-10,1.205670e-09,1.569920e-09,1.437969e-09,1.264825e-09,...,7.596838e-10,1.185676e-09,1.408344e-09,9.396106e-10,3.263996e-10,8.116416e-10,8.458283e-10,1.133015e-09,1.393282e-09,5.907058e-10
2,Frontiers_CP-66_CP-66_20210716-000000,pwr_lin,-2.655856e-09,-1.179579e-08,-3.378029e-08,-2.047574e-08,7.866821e-08,1.499196e-07,3.369719e-07,6.925233e-07,...,1.326742e-06,4.334578e-07,-5.060552e-07,-8.149085e-07,-1.776411e-06,-2.887063e-06,-2.095681e-06,-8.204266e-07,-1.517324e-07,3.144448e-07
3,Frontiers_CP-66_CP-66_20210716-000000,pwr_rot,-7.878670e-08,-2.514187e-07,-1.262197e-07,-1.130888e-07,-2.835921e-07,-3.726526e-07,-3.114805e-07,-3.268763e-07,...,7.192581e-08,7.764132e-08,-1.740640e-08,-1.393403e-07,-1.414921e-08,9.265420e-08,-4.781753e-08,1.226706e-07,-8.519107e-08,-1.559372e-07
4,Frontiers_CP-66_CP-66_20210716-000000,rsp,-4.379060e-01,-4.374503e-01,-4.365397e-01,-4.352323e-01,-4.334734e-01,-4.313219e-01,-4.287250e-01,-4.257420e-01,...,7.997676e-01,7.925788e-01,7.853790e-01,7.781603e-01,7.709196e-01,7.636476e-01,7.563429e-01,7.489946e-01,7.416031e-01,7.341561e-01


In [4]:
df_2.head()

,record_id,study_id,subject_id,sex,height,weight,age,hf_type
0,Frontiers_CP-66_CP-66_20210716-000000,Frontiers,CP-66,Male,1.650,66.0,59.0,UNKNOWN
1,Frontiers_CP-59_CP-59_20210716-000000,Frontiers,CP-59,Female,1.580,56.5,57.0,UNKNOWN
2,Frontiers_CP-61_CP-61_20210716-000000,Frontiers,CP-61,Female,1.640,83.0,68.0,UNKNOWN
3,Frontiers_UP-03_UP-03_20180404-000000,Frontiers,UP-03,Male,1.625,66.0,96.0,UNKNOWN
4,Frontiers_CP-57_CP-57_20210716-000000,Frontiers,CP-57,Male,1.740,62.0,79.0,UNKNOWN


In [5]:
df_1 = df_1.sample(n=500)

In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 523 to 3712
Columns: 12003 entries, record_id to 0 days 00:01:00
dtypes: float64(12001), object(2)
memory usage: 45.8+ MB


In [7]:
merged_df = pd.merge(df_1, df_2, on='record_id')

In [8]:
merged_df.head()

,record_id,channel,0 days 00:00:00,0 days 00:00:00.005000,0 days 00:00:00.010000,0 days 00:00:00.015000,0 days 00:00:00.020000,0 days 00:00:00.025000,0 days 00:00:00.030000,0 days 00:00:00.035000,...,0 days 00:00:59.990000,0 days 00:00:59.995000,0 days 00:01:00,study_id,subject_id,sex,height,weight,age,hf_type
0,HAP-PWV_AR_0_20220216-104722_14xreOAagSOvTo,pwr_rot,2.725892e-04,2.600196e-04,-4.677789e-03,-1.442255e-02,-2.304427e-02,-1.823420e-02,-1.231658e-02,-7.787862e-03,...,NaN,NaN,NaN,HAP-PWV,AR,Female,1.65,83.0,70.0,NoHF
1,HAP-PWV_AR_0_20220216-104722_14xreOAagSOvTo,pwr_lin,-3.447429e-03,-2.436293e-02,-4.329472e-02,-5.436322e-02,-5.184779e-02,-4.038738e-02,-3.335469e-02,-2.847132e-02,...,NaN,NaN,NaN,HAP-PWV,AR,Female,1.65,83.0,70.0,NoHF
2,SLEEP-SIMUL_WN_Centrale_20181116-135820_Dy2O8z...,pwr_rot,3.003260e-22,-1.521833e-07,-4.906979e-07,-5.393484e-07,-1.652647e-07,3.358045e-07,6.957038e-07,6.029253e-07,...,-2.915410e-06,-6.610624e-07,1.350023e-06,SLEEP-SIMUL,WN,Female,1.83,83.0,22.0,NoHF
3,HAP-Test_MC_1_20220214-095918_nWY6q10DoiKSD,pwr_rot,2.122173e-04,3.900320e-04,-2.762427e-03,-9.362706e-03,-1.624580e-02,-1.480981e-02,-1.158942e-02,-8.247929e-03,...,NaN,NaN,NaN,HAP-Test,MC,Female,1.60,68.0,69.0,NoHF
4,SLEEP-SIMUL_PV_Valsalva_20181108-133432_XzE6DB...,pwr_lin,-8.528647e-26,2.438460e-07,1.063671e-06,1.554705e-06,8.642172e-07,-6.867579e-07,-2.304062e-06,-3.017287e-06,...,1.366741e-07,3.263732e-07,3.719346e-07,SLEEP-SIMUL,PV,Female,1.65,65.0,21.0,NoHF


In [9]:
merged_df['hf_type'].value_counts()

NoHF       263
UNKNOWN    169
HFrEF       46
HFpEF       12
HFmEF       10
Name: hf_type, dtype: int64

<h1>Deep Learning Classifiers</h1>

In [10]:
import pandas as pd
import numpy as np

# Merge your dataframes here
merged_df = pd.merge(df_1, df_2, on='record_id')

# Generate some example time series data
data = merged_df.select_dtypes(include=[np.float64]).values

# Reshape the data to fit the input shape of the model
num_samples, num_timesteps = data.shape
data = data.reshape(num_samples, num_timesteps, 1)





In [11]:
!pip install tensorflow==2.0.0 --quiet

ERROR: Could not find a version that satisfies the requirement tensorflow==2.0.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.0.0


<h1>CNN</h1>

In [12]:
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping # import EarlyStopping callback
import pandas as pd
import numpy as np

# Convert target variable to binary vector
y = np.array(merged_df['hf_type'])
y = np.expand_dims(y, axis=1)
y = (y == np.arange(2)).astype(int)
y = np.argmax(y, axis=1)

n_timesteps = data.shape[1]
n_features = data.shape[2]

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # changed number of units and activation function

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

# Add EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Fit the model with early stopping
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])










2023-04-02 11:22:40.263824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-02 11:22:40.263851: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-02 11:22:40.263870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (brandon-HP-255-G7-Notebook-PC): /proc/driver/nvidia/version does not exist
2023-04-02 11:22:40.264092: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
13/13 [==============================] - 20s 1s/step - loss: nan - accuracy: 0.9450 - val_loss: nan - val_accuracy: 1.0000
Epoch 2/50
13/13 [==============================] - 18s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 3/50
13/13 [==============================] - 18s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 4/50
13/13 [==============================] - 18s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 5/50
13/13 [==============================] - 18s 1s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000


<h1>LSTM</h1>

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Convert target variable to binary vector
y = np.array(merged_df['hf_type'])
y = np.expand_dims(y, axis=1)
y = (y == np.arange(2)).astype(int)
y = np.argmax(y, axis=1)

model = Sequential()
model.add(LSTM(64, input_shape=(n_timesteps, n_features), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

# Define early stopping callback
early_stopping = EarlyStopping(patience=1, monitor='val_loss')

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/20
13/13 [==============================] - 453s 34s/step - loss: nan - accuracy: 0.9950 - val_loss: nan - val_accuracy: 1.0000
Epoch 2/20
13/13 [==============================] - 462s 36s/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000


<h1>Multilayer Perceptron</h1>

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.callbacks import EarlyStopping

# Generate dummy data
n_samples, n_timesteps, n_features = 400, 10, 5
X = np.random.rand(n_samples, n_timesteps, n_features)
y = np.random.randint(2, size=n_samples)

# Flatten the input data
X_flat = np.reshape(X, (n_samples, n_timesteps * n_features))

# Split the data into training and testing sets
X_train_flat, X_test_flat, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)

# Define the MLP model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=n_timesteps * n_features))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(patience=3, monitor='val_loss')

# Train the model with early stopping
history = model.fit(X_train_flat, y_train, epochs=50, batch_size=32, validation_data=(X_test_flat, y_test), callbacks=[early_stopping])






Epoch 1/50
10/10 [==============================] - 1s 23ms/step - loss: 0.7024 - accuracy: 0.5031 - val_loss: 0.7047 - val_accuracy: 0.4875
Epoch 2/50
10/10 [==============================] - 0s 7ms/step - loss: 0.6910 - accuracy: 0.5125 - val_loss: 0.7087 - val_accuracy: 0.4500
Epoch 3/50
10/10 [==============================] - 0s 6ms/step - loss: 0.6850 - accuracy: 0.5625 - val_loss: 0.7079 - val_accuracy: 0.4875
Epoch 4/50
10/10 [==============================] - 0s 7ms/step - loss: 0.6798 - accuracy: 0.5813 - val_loss: 0.7112 - val_accuracy: 0.4750


<h1>Feed Forward Neural Network</h1>

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np



# Convert non-numeric values to NaN
merged_df = merged_df.apply(pd.to_numeric, errors='coerce')

# Replace NaN with a specific value (e.g., 0)
merged_df = merged_df.fillna(0)

# Split data into features and target
X = merged_df.drop(['hf_type'], axis=1).values.astype('float32')
y = merged_df['hf_type'].values

# Convert target variable to binary vector
y = (y == 1).astype(int)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create FNN model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(patience=5, monitor='val_loss')

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])







Epoch 1/50
13/13 [==============================] - 1s 28ms/step - loss: 0.9497 - accuracy: 0.6475 - val_loss: 0.1087 - val_accuracy: 0.9900
Epoch 2/50
13/13 [==============================] - 0s 16ms/step - loss: 0.2567 - accuracy: 0.9325 - val_loss: 0.0109 - val_accuracy: 1.0000
Epoch 3/50
13/13 [==============================] - 0s 16ms/step - loss: 0.2065 - accuracy: 0.9725 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 4/50
13/13 [==============================] - 0s 19ms/step - loss: 0.1601 - accuracy: 0.9800 - val_loss: 6.0186e-04 - val_accuracy: 1.0000
Epoch 5/50
13/13 [==============================] - 0s 16ms/step - loss: 0.1671 - accuracy: 0.9825 - val_loss: 2.3036e-04 - val_accuracy: 1.0000
Epoch 6/50
13/13 [==============================] - 0s 15ms/step - loss: 0.0929 - accuracy: 0.9925 - val_loss: 1.0804e-04 - val_accuracy: 1.0000
Epoch 7/50
13/13 [==============================] - 0s 15ms/step - loss: 0.0423 - accuracy: 0.9900 - val_loss: 6.0306e-05 - val_accuracy: 1.00

<h1>Decision Tree Classifier</h1>

In [16]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Define features and target variable
features = ["age", "weight", "height"]
target = "hf_type"

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    merged_df[features], merged_df[target], test_size=0.2, random_state=42
)

# Initialize classifier
clf = DecisionTreeClassifier(random_state=42)

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate f1 score
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 score:", f1)





Accuracy: 1.0
F1 score: 1.0


<h1>Random Forest Classifier</h1>

In [17]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate a random dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_classes=2, random_state=42)

# Reshape y to a 1D array
y = np.ravel(y)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a random forest classifier with 100 trees
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier to the training data
clf.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# Predict on the test set
y_pred = clf.predict(X_test.reshape(X_test.shape[0], -1))

# Print the accuracy of the classifier
print(f"Accuracy: {clf.score(X_test.reshape(X_test.shape[0], -1), y_test)}")




Accuracy: 0.945


In [18]:
!pip install streamlit --quiet
!pip install cachetools --quiet


In [27]:
!pip3 install cachetools==5.3.0

  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.4
    Uninstalling cachetools-4.2.4:
      Successfully uninstalled cachetools-4.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth 2.3.3 requires cachetools<5.0,>=2.0.0, but you have cachetools 5.3.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import streamlit as st

# Define features and target variable
features = ["age", "weight", "height"]
target = "hf_type"

# Initialize classifier
clf = DecisionTreeClassifier(random_state=42)

# Define the app
def app():
    # Add Omdena logo
    st.image("omdena_logo.png")

    # Add a title
    st.title("Heart Failure Risk Detector")

    # Add input fields for age, weight, and height
    age = st.number_input("Enter age", value=40, min_value=0, max_value=120)
    weight = st.number_input("Enter weight in kg", value=70, min_value=0, max_value=500)
    height = st.number_input("Enter height in cm", value=170, min_value=0, max_value=300)

    # Create a dataframe with the user's input
    data = pd.DataFrame({
        "age": [age],
        "weight": [weight],
        "height": [height]
    })

    # Make a prediction using the classifier
    prediction = clf.predict(data)

    # Add a button to make the prediction
    if st.button("Predict"):
        # Show the prediction
        if prediction == 0:
            st.error("You are at risk of heart failure!")
        else:
            st.success("You are not at risk of heart failure.")

# Run the app
if __name__ == '__main__':
    app()




In [ ]:
streamlit run app.py


<h1>Experimental Dashboard w/Decision Tree</h1>

In [ ]:
#Remember to replace the path with your own path and feel free to change the name of the dashboard.html to a style of your choice

from flask import Flask, render_template, request
from wtforms import Form, StringField, validators
import pandas as pd

# Define the Flask app
app = Flask(__name__, template_folder='/home/brandon/Omdena/Omdena HeartKinetics/HeartKinetics - Project/Dataset1/templates')


# Define the input form
class InputForm(Form):
    input_data = StringField('Input Data', [validators.DataRequired()])

# Define the dashboard route
@app.route('/', methods=['GET', 'POST'])
def dashboard():
    form = InputForm(request.form)
    result = None
    if request.method == 'POST' and form.validate():
        # Get the input data from the form
        input_data = form.input_data.data
        
        # Perform some processing on the input data
        processed_data = input_data.upper()
        
        # Create a DataFrame from the processed data
        df = pd.DataFrame({'Processed Data': [processed_data]})
        
        # Set the DataFrame as the result
        result = df.to_html(index=False)
        
    return render_template('dashboard.html', form=form, result=result)

# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)



